# Template extraction #

Parse text documents which are based on a template and stored in an excel document

Install dependencies and import the nlp_datau package

In [1]:
import yaml
!pip install -r ./../requirements.txt
import sys
sys.path.append("./../")


## 1. Extract substring of substring ##

Set variables

In [2]:
path_excel_sheet_in = '/home/jovyan/work/E/XML/19/report.xlsx'
path_excel_sheet_out = '/home/jovyan/work/E/XML/19/report+nevenbevindingen.xlsx'

# path_excel_sheet_in = './../resources/data_example/standaardverslag_covid.xlsx'
# path_excel_sheet_out = './../resources/data_example/standaardverslag_covid_out.xlsx'

column_source = 'text'
column_target_conclusion = 'conclusion'
column_target_incidental_findings = 'incidental_findings'
column_target_missing_incidental_findings = 'missing_incidental_findings'
column_corads = 'corads'

regex_conclusion = r"Conclusie:(.*?)(DISCLAIMER|Met collegiale groet)"
regex_incidental_findings = r"(?<=Nevenbevindingen:)(.*?)(?=(DISCLAIMER|Met collegiale groet))"

import pandas
from IPython.core.display import display
df = pandas.read_excel(path_excel_sheet_in, header=0)
display(df)

,text
0,Blanco CT Thorax \nCOVID 19 Standaardverslag \...


Extract findings

In [3]:
from nlp_datau.string_utils import select_regex, regex_missing, normalize_regex

df[column_target_conclusion] = df[column_source].apply(lambda x:select_regex(x, regex=regex_conclusion))
df[column_target_incidental_findings] = df[column_target_conclusion].apply(lambda x:select_regex(x, regex=regex_incidental_findings))
df[column_target_missing_incidental_findings] = df[column_target_conclusion].apply(lambda x:regex_missing(x, regex=regex_incidental_findings))


Extract corads

In [4]:
corads_config = './../resources/config/pycontextnlp_tnm_targets_corads_nl.yml'

stream = open(corads_config, "r")
targets = yaml.load_all(stream)

df[column_corads] = df[column_target_conclusion].apply(lambda x:normalize_regex(x, targets=targets))
display(df)

C:\git\nlp-datau\nlp_datau\string_utils.py:23: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  for target in targets:


,text,conclusion,incidental_findings,missing_incidental_findings,corads
0,Blanco CT Thorax \nCOVID 19 Standaardverslag \...,Conclusie:Beeld [niet passend bij / passend bi...,{ Dit is een hele vreemde bevinding },False,


Run and write to out file

In [5]:
df.to_excel(path_excel_sheet_out)